# Step 1
## To use a Git repository with Snowflake, follow these steps:
#### 1) Create a secret, if needed, to contain credentials for authenticating with the repository.
#### 2) Create an API integration to specify details about Snowflake interaction with the Git repository API.
#### 3) Create a Snowflake Git repository stage to which you can synchronize files from the repository.

We are currently in a holding pattern as I do not have access to set a Espresso private key token.  I've asked Huseyin to set it up and put the result into Bitwarden.

Resources:
Git Token:  https://docs.github.com/en/organizations/managing-programmatic-access-to-your-organization/setting-a-personal-access-token-policy-for-your-organization#restricting-access-by-personal-access-tokens

Snowflake Setup:  https://docs.snowflake.com/en/developer-guide/git/git-setting-up

Snowflake Connecting:  https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-snowgit

In [ ]:
-- going to set this up for personal access first to my repository dhagenespressocapitalcom/snowflake_notebooks_test
-- USE ROLE securityadmin;
-- CREATE ROLE myco_secrets_admin;
-- GRANT CREATE SECRET ON SCHEMA myco_db.integrations TO ROLE myco_secrets_admin;

-- USE ROLE myco_db_owner;
-- GRANT USAGE ON DATABASE myco_db TO ROLE myco_secrets_admin;
-- GRANT USAGE ON SCHEMA myco_db.integrations TO ROLE myco_secrets_admin;

-- USE ROLE myco_secrets_admin;
-- USE DATABASE myco_db;
-- USE SCHEMA myco_db.integrations;

-- CREATE OR REPLACE SECRET myco_git_secret
--   TYPE = password
--   USERNAME = 'gladyskravitz'
--   PASSWORD = 'ghp_token';

-- the following is for testing only until we can setup full account level access.

USE ROLE accountadmin;
CREATE ROLE notebooks_secrets_admin;
GRANT CREATE SECRET ON SCHEMA analytics.user_objects TO ROLE notebooks_secrets_admin;

-- -- USE ROLE bi_analyst;
-- -- GRANT USAGE ON DATABASE myco_db TO ROLE notebooks_secrets_admin;
-- -- GRANT USAGE ON SCHEMA myco_db.integrations TO ROLE notebooks_secrets_admin;

-- =========================== NOTE ============================================================== --
-- note:  might be able to grant the role to the bi_analyst role and avoid user grants here. --
GRANT ROLE notebooks_secrets_admin TO USER dhagenespressocapitalcom;

USE ROLE notebooks_secrets_admin;
USE DATABASE analytics;
USE SCHEMA analytics.user_objects;

CREATE OR REPLACE SECRET dkh_git_secret
  TYPE = password
  USERNAME = '--not here--'
  PASSWORD = '--not here--';

## Create an API Integration

In [ ]:

USE ROLE accountadmin;
GRANT CREATE INTEGRATION ON ACCOUNT TO ROLE notebooks_secrets_admin;

USE ROLE notebooks_secrets_admin;
USE DATABASE analytics;
USE SCHEMA analytics.user_objects;

CREATE OR REPLACE API INTEGRATION dkh_git_api_integration
  API_PROVIDER = git_https_api
  API_ALLOWED_PREFIXES = ('https://github.com/dhagenespressocapitalcom')
  ALLOWED_AUTHENTICATION_SECRETS = (dkh_git_secret)
  ENABLED = TRUE;

# Create the repository

In [ ]:
USE ROLE accountadmin;
GRANT CREATE GIT REPOSITORY ON SCHEMA analytics.user_objects TO ROLE notebooks_secrets_admin;

USE ROLE notebooks_secrets_admin;

CREATE OR REPLACE GIT REPOSITORY snowflake_notebooks_test
  API_INTEGRATION = dkh_git_api_integration
  GIT_CREDENTIALS = dkh_git_secret
  ORIGIN = 'https://github.com/dhagenespressocapitalcom/snowflake_notebooks_test.git';

In [ ]:
USE ROLE accountadmin;
GRANT CREATE GIT REPOSITORY ON SCHEMA analytics.user_objects TO ROLE notebooks_secrets_admin;

DROP GIT REPOSITORY snowflake_extensions;